# Universal SEO Agent Graph (LangGraph + CrewAI + LangChain)

This notebook implements a **professional, production-ready SEO AI agent graph** that works seamlessly across **LangGraph** and **CrewAI** frameworks, powered by **LangChain tools**.

## Architecture Overview

- **State Management**: Typed, shared state carrying inputs → findings → plan → execution → validation
- **LangChain Tools**: API connectors (GSC, GA4, crawlers, SERP, backlinks)
- **LangGraph Nodes + Edges**: Deterministic orchestration with conditional routing, approvals, and persistence
- **CrewAI Agents + Tasks**: Hierarchical specialists delegating SEO work
- **Framework Adapters**: Unified interface for both frameworks

---

In [ ]:
# STEP 1: Install & Import Required Libraries
# =============================================================================

import subprocess
import sys

# Install required packages
packages = [
    'langchain==0.1.14',
    'langchain-core==0.1.34',
    'langgraph==0.0.41',
    'crewai==0.20.0',
    'pydantic==2.5.3',
    'python-dotenv==1.0.0',
    'requests==2.31.0'
]

for package in packages:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', package])

print("✓ All dependencies installed successfully")

In [ ]:
# Import Core Libraries
from typing import TypedDict, List, Dict, Optional, Any, Callable, Union
from dataclasses import dataclass, field
from enum import Enum
import json
from datetime import datetime
import uuid

# LangChain imports
from langchain.tools import tool, StructuredTool, Tool
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.agents import Agent, AgentExecutor

# LangGraph imports
from langgraph.graph import StateGraph, END
from langgraph.graph.graph import CompiledGraph

print("✓ Core imports successful")

In [ ]:
# STEP 2: Define Enums & Structured Output Types
# =============================================================================

class IssueType(str, Enum):
    """Categorizes SEO issues by domain."""
    TECH_SEO = "TECH_SEO"
    PRODUCT_SEO = "PRODUCT_SEO"
    CONTENT = "CONTENT"
    AUTHORITY = "AUTHORITY"
    UNKNOWN = "UNKNOWN"

class IssueSeverity(str, Enum):
    """Risk levels for issues."""
    CRITICAL = "CRITICAL"
    HIGH = "HIGH"
    MEDIUM = "MEDIUM"
    LOW = "LOW"

class ExecutionStatus(str, Enum):
    """Tracks execution progress."""
    PENDING = "PENDING"
    APPROVED = "APPROVED"
    EXECUTING = "EXECUTING"
    COMPLETED = "COMPLETED"
    FAILED = "FAILED"
    ROLLED_BACK = "ROLLED_BACK"


@dataclass
class Issue:
    """Represents a detected SEO issue."""
    issue_id: str = field(default_factory=lambda: str(uuid.uuid4()))
    title: str = ""
    description: str = ""
    issue_type: IssueType = IssueType.UNKNOWN
    severity: IssueSeverity = IssueSeverity.LOW
    affected_urls: List[str] = field(default_factory=list)
    source: str = ""  # GSC, GA4, Crawl, SERP
    detected_at: str = field(default_factory=lambda: datetime.now().isoformat())
    
    def to_dict(self) -> Dict:
        return {
            'issue_id': self.issue_id,
            'title': self.title,
            'description': self.description,
            'issue_type': self.issue_type.value,
            'severity': self.severity.value,
            'affected_urls': self.affected_urls,
            'source': self.source,
            'detected_at': self.detected_at
        }


@dataclass
class Opportunity:
    """Represents a potential SEO improvement."""
    opportunity_id: str = field(default_factory=lambda: str(uuid.uuid4()))
    title: str = ""
    description: str = ""
    opportunity_type: IssueType = IssueType.UNKNOWN
    impact: float = 0.0  # 0-10 scale
    confidence: float = 0.0  # 0-10 scale
    effort: float = 0.0  # 0-10 scale
    impact_score: float = 0.0  # (Impact * Confidence) / Effort
    target_urls: List[str] = field(default_factory=list)
    
    def to_dict(self) -> Dict:
        return {
            'opportunity_id': self.opportunity_id,
            'title': self.title,
            'description': self.description,
            'opportunity_type': self.opportunity_type.value,
            'impact': self.impact,
            'confidence': self.confidence,
            'effort': self.effort,
            'impact_score': self.impact_score,
            'target_urls': self.target_urls
        }


@dataclass
class ChangeSet:
    """Represents a proposed or executed change."""
    changeset_id: str = field(default_factory=lambda: str(uuid.uuid4()))
    url: str = ""
    change_type: str = ""  # meta_tag, schema, redirect, content, etc.
    before: str = ""
    after: str = ""
    rollback_plan: str = ""
    risk_level: IssueSeverity = IssueSeverity.LOW
    status: ExecutionStatus = ExecutionStatus.PENDING
    created_at: str = field(default_factory=lambda: datetime.now().isoformat())
    
    def to_dict(self) -> Dict:
        return {
            'changeset_id': self.changeset_id,
            'url': self.url,
            'change_type': self.change_type,
            'before': self.before,
            'after': self.after,
            'rollback_plan': self.rollback_plan,
            'risk_level': self.risk_level.value,
            'status': self.status.value,
            'created_at': self.created_at
        }


@dataclass
class Ticket:
    """Represents an actionable work item (e.g., Jira ticket)."""
    ticket_id: str = field(default_factory=lambda: str(uuid.uuid4()))
    title: str = ""
    description: str = ""
    assignee: str = ""
    priority: IssueSeverity = IssueSeverity.MEDIUM
    related_issue_ids: List[str] = field(default_factory=list)
    related_changesets: List[str] = field(default_factory=list)
    status: str = "TODO"
    created_at: str = field(default_factory=lambda: datetime.now().isoformat())
    
    def to_dict(self) -> Dict:
        return {
            'ticket_id': self.ticket_id,
            'title': self.title,
            'description': self.description,
            'assignee': self.assignee,
            'priority': self.priority.value,
            'related_issue_ids': self.related_issue_ids,
            'related_changesets': self.related_changesets,
            'status': self.status,
            'created_at': self.created_at
        }


print("✓ Structured output types defined")

In [ ]:
# STEP 3: Define Shared State Schema (LangGraph StateGraph)
# =============================================================================

class SEOAgentState(TypedDict, total=False):
    """
    Universal state object carrying all data through the graph pipeline.
    This is the "single source of truth" for a run.
    """
    # Run metadata
    run_id: str
    created_at: str
    
    # Input snapshots (raw data)
    inputs: Dict[str, Any]  # GSC/GA4/crawl/SERP/backlinks snapshots
    
    # Normalized findings
    findings: Dict[str, Any]  # normalized issues + opportunities
    issues: List[Dict[str, Any]]  # Issue objects
    opportunities: List[Dict[str, Any]]  # Opportunity objects
    
    # Scoring & prioritization
    scores: Dict[str, float]  # Impact × Confidence ÷ Effort + risk
    
    # Plan (what we're going to do)
    plan: Dict[str, Any]  # chosen actions, changesets, tickets
    changesets: List[Dict[str, Any]]  # ChangeSet objects
    tickets: List[Dict[str, Any]]  # Ticket objects
    
    # Approval tracking
    approvals: Dict[str, Any]  # status + reviewer notes
    requires_approval: bool
    approval_status: str  # PENDING, APPROVED, REJECTED
    approval_notes: str
    
    # Execution details
    execution: Dict[str, Any]  # what was applied + where
    executed_changesets: List[Dict[str, Any]]
    execution_errors: List[str]
    
    # Validation & metrics
    validation: Dict[str, Any]  # before/after metrics + checks
    metrics_before: Dict[str, float]
    metrics_after: Dict[str, float]
    
    # Audit log (full traceability)
    audit_log: List[Dict[str, Any]]
    
    # Error tracking
    errors: List[str]
    
    # Routing state
    current_route: str
    next_node: str


def initialize_state(run_id: Optional[str] = None) -> SEOAgentState:
    """Initialize a fresh state object."""
    return {
        'run_id': run_id or str(uuid.uuid4()),
        'created_at': datetime.now().isoformat(),
        'inputs': {},
        'findings': {},
        'issues': [],
        'opportunities': [],
        'scores': {},
        'plan': {},
        'changesets': [],
        'tickets': [],
        'approvals': {},
        'requires_approval': False,
        'approval_status': 'PENDING',
        'approval_notes': '',
        'execution': {},
        'executed_changesets': [],
        'execution_errors': [],
        'validation': {},
        'metrics_before': {},
        'metrics_after': {},
        'audit_log': [],
        'errors': [],
        'current_route': 'START',
        'next_node': 'collect_inputs'
    }


def log_to_audit(state: SEOAgentState, action: str, details: Dict) -> SEOAgentState:
    """Add an entry to the audit log."""
    log_entry = {
        'timestamp': datetime.now().isoformat(),
        'action': action,
        'details': details,
        'state_snapshot': {
            'run_id': state.get('run_id'),
            'current_route': state.get('current_route')
        }
    }
    if 'audit_log' not in state:
        state['audit_log'] = []
    state['audit_log'].append(log_entry)
    return state


print("✓ State schema defined")

In [ ]:
# STEP 4: Implement LangChain Tools (SEO Data Connectors)
# =============================================================================

# Tool 1: Google Search Console Connector
@tool
def collect_gsc_data(domain: str, days: int = 30) -> Dict[str, Any]:
    """
    Fetch performance data from Google Search Console.
    Returns clicks, impressions, CTR, position by query and page.
    """
    # MOCK IMPLEMENTATION - Replace with actual GSC API
    return {
        'domain': domain,
        'period_days': days,
        'top_queries': [
            {'query': 'best laptop', 'clicks': 150, 'impressions': 2000, 'ctr': 0.075, 'position': 3.2},
            {'query': 'laptop reviews', 'clicks': 80, 'impressions': 1200, 'ctr': 0.067, 'position': 4.1}
        ],
        'pages_with_issues': [
            {'url': '/products/laptop-x1', 'clicks': 0, 'impressions': 500, 'issue': 'not_indexed'},
            {'url': '/blog/best-laptops', 'clicks': 20, 'impressions': 800, 'issue': 'low_ctr'}
        ]
    }


# Tool 2: Google Analytics 4 Connector
@tool
def collect_ga4_data(property_id: str, days: int = 30) -> Dict[str, Any]:
    """
    Fetch user engagement metrics from Google Analytics 4.
    Returns sessions, users, bounce rate, conversion data.
    """
    # MOCK IMPLEMENTATION - Replace with actual GA4 API
    return {
        'property_id': property_id,
        'period_days': days,
        'sessions': 15000,
        'users': 8000,
        'bounce_rate': 0.45,
        'avg_session_duration': 180,
        'conversions': 150,
        'conversion_rate': 0.01,
        'pages_by_traffic': [
            {'page': '/products', 'sessions': 5000, 'users': 3000, 'bounce_rate': 0.35},
            {'page': '/blog', 'sessions': 3000, 'users': 2000, 'bounce_rate': 0.55}
        ]
    }


# Tool 3: Website Crawler (simulated)
@tool
def crawl_website(domain: str, max_pages: int = 100) -> Dict[str, Any]:
    """
    Crawl website to detect on-page SEO issues.
    Returns indexability, meta tags, schema markup, duplicates, etc.
    """
    # MOCK IMPLEMENTATION - Replace with actual crawler (Screaming Frog API, etc)
    return {
        'domain': domain,
        'total_pages_crawled': 87,
        'issues': {
            'missing_meta_descriptions': 12,
            'duplicate_titles': 3,
            'missing_h1': 5,
            'broken_links': 8,
            'missing_schema': 15,
            'slow_pages': 7,
            'mobile_usability_errors': 4
        },
        'sample_issues': [
            {'url': '/products/laptop', 'issue': 'missing_schema', 'severity': 'high'},
            {'url': '/blog/article', 'issue': 'slow_pages', 'severity': 'medium'}
        ]
    }


# Tool 4: SERP Snapshot
@tool
def collect_serp_snapshot(keywords: List[str], location: str = "US") -> Dict[str, Any]:
    """
    Capture current SERP positions for tracked keywords.
    Returns ranking positions, featured snippets, PAA, etc.
    """
    # MOCK IMPLEMENTATION - Replace with actual SERP tracking API
    return {
        'keywords': keywords,
        'location': location,
        'snapshot_date': datetime.now().isoformat(),
        'rankings': [
            {'keyword': 'best laptop', 'position': 3, 'featured_snippet': False, 'url': 'example.com/products/laptop'},
            {'keyword': 'laptop reviews', 'position': 5, 'featured_snippet': True, 'url': 'example.com/blog/reviews'}
        ],
        'paa': ['best laptops 2024', 'laptop vs desktop', 'gaming laptop reviews']
    }


# Tool 5: Backlink Profile
@tool
def collect_backlinks(domain: str) -> Dict[str, Any]:
    """
    Fetch backlink profile and authority metrics.
    Returns referring domains, anchor texts, DR/UR, toxic links.
    """
    # MOCK IMPLEMENTATION - Replace with actual backlink API (Ahrefs, Semrush)
    return {
        'domain': domain,
        'total_backlinks': 2500,
        'referring_domains': 450,
        'domain_rating': 42,
        'url_rating': 35,
        'toxic_links': 15,
        'top_referring_domains': [
            {'domain': 'site1.com', 'backlinks': 45, 'dr': 65},
            {'domain': 'site2.com', 'backlinks': 32, 'dr': 58}
        ]
    }


# Tool 6: Normalize & Classify Issues
@tool
def normalize_inputs(gsc: Dict, ga4: Dict, crawl: Dict, serp: Dict, backlinks: Dict) -> Dict[str, List]:
    """
    Aggregate and normalize issues from all sources into a unified schema.
    """
    issues = []
    opportunities = []
    
    # Extract issues from crawl data
    for issue_type, count in crawl.get('issues', {}).items():
        if count > 0:
            issues.append(Issue(
                title=f"Found {count} pages with {issue_type}",
                description=f"{count} pages detected with {issue_type} issue",
                issue_type=IssueType.TECH_SEO,
                severity=IssueSeverity.HIGH if count > 5 else IssueSeverity.MEDIUM,
                source="Crawl"
            ).to_dict())
    
    # Extract opportunities from GSC data
    for query_data in gsc.get('top_queries', [])[:3]:
        if query_data['ctr'] < 0.05:  # Low CTR
            opportunities.append(Opportunity(
                title=f"Improve meta for '{query_data['query']}'",
                description=f"Query '{query_data['query']}' has low CTR ({query_data['ctr']*100:.1f}%)",
                opportunity_type=IssueType.PRODUCT_SEO,
                impact=7.0,
                confidence=8.0,
                effort=2.0
            ).to_dict())
    
    return {'issues': issues, 'opportunities': opportunities}


print("✓ LangChain tools implemented")

In [ ]:
# STEP 5: Define LangGraph Nodes (Graph Stages)
# =============================================================================

def node_collect_inputs(state: SEOAgentState) -> SEOAgentState:
    """
    COLLECT NODE: Gather snapshots from all SEO data sources.
    Executes: collect_gsc, collect_ga4, crawl_website, collect_serp, collect_backlinks.
    """
    print(f"[COLLECT] Run {state['run_id']}: Gathering SEO data...")
    
    # Execute tools
    gsc = collect_gsc_data(domain="example.com", days=30)
    ga4 = collect_ga4_data(property_id="G-XXXXXX", days=30)
    crawl = crawl_website(domain="example.com", max_pages=100)
    serp = collect_serp_snapshot(keywords=["best laptop", "laptop reviews"], location="US")
    backlinks = collect_backlinks(domain="example.com")
    
    state['inputs'] = {
        'gsc': gsc,
        'ga4': ga4,
        'crawl': crawl,
        'serp': serp,
        'backlinks': backlinks
    }
    
    state = log_to_audit(state, "collect_inputs", {"tools_executed": 5})
    state['current_route'] = "collected"
    state['next_node'] = "normalize_inputs"
    
    return state


def node_normalize_inputs(state: SEOAgentState) -> SEOAgentState:
    """
    NORMALIZE NODE: Standardize inputs into unified schema (Issue, Opportunity).
    Validates: all inputs present, no duplicates, proper typing.
    """
    print(f"[NORMALIZE] Run {state['run_id']}: Normalizing findings...")
    
    inputs = state.get('inputs', {})
    normalized = normalize_inputs(
        gsc=inputs.get('gsc', {}),
        ga4=inputs.get('ga4', {}),
        crawl=inputs.get('crawl', {}),
        serp=inputs.get('serp', {}),
        backlinks=inputs.get('backlinks', {})
    )
    
    state['issues'] = normalized['issues']
    state['opportunities'] = normalized['opportunities']
    state['findings'] = {
        'total_issues': len(normalized['issues']),
        'total_opportunities': len(normalized['opportunities'])
    }
    
    state = log_to_audit(state, "normalize_inputs", {"issues": len(state['issues']), "opportunities": len(state['opportunities'])})
    state['current_route'] = "normalized"
    state['next_node'] = "score_opportunities"
    
    return state


def node_score_opportunities(state: SEOAgentState) -> SEOAgentState:
    """
    SCORE NODE: Prioritize issues & opportunities using Impact × Confidence / Effort formula.
    Filters: only include Impact Score > threshold.
    """
    print(f"[SCORE] Run {state['run_id']}: Scoring opportunities...")
    
    # Score opportunities
    scored = []
    for opp in state.get('opportunities', []):
        impact_score = (opp['impact'] * opp['confidence']) / max(opp['effort'], 1)
        opp['impact_score'] = impact_score
        scored.append(opp)
    
    # Sort by impact_score descending
    scored = sorted(scored, key=lambda x: x['impact_score'], reverse=True)
    state['opportunities'] = scored
    
    # Build scores dict
    state['scores'] = {
        f"opp_{i}": opp['impact_score'] for i, opp in enumerate(scored[:5])
    }
    
    state = log_to_audit(state, "score_opportunities", {"top_score": state['scores'].get('opp_0', 0)})
    state['current_route'] = "scored"
    state['next_node'] = "route_by_type"
    
    return state


def node_route_by_type(state: SEOAgentState) -> SEOAgentState:
    """
    ROUTE NODE: Determine which specialist agent should handle this.
    Output: sets next_node based on issue/opportunity type.
    """
    print(f"[ROUTE] Run {state['run_id']}: Routing to specialists...")
    
    # Determine dominant issue type
    issues = state.get('issues', [])
    if not issues:
        state['next_node'] = "qa_guardrail"
        return state
    
    # Count by type
    type_counts = {}
    for issue in issues:
        issue_type = issue.get('issue_type', 'UNKNOWN')
        type_counts[issue_type] = type_counts.get(issue_type, 0) + 1
    
    dominant_type = max(type_counts.items(), key=lambda x: x[1])[0] if type_counts else "UNKNOWN"
    
    # Route to appropriate specialist
    routing_map = {
        'TECH_SEO': 'techseo_agent_node',
        'PRODUCT_SEO': 'productseo_agent_node',
        'CONTENT': 'content_agent_node',
        'AUTHORITY': 'authority_agent_node',
        'UNKNOWN': 'qa_guardrail'
    }
    
    state['current_route'] = dominant_type
    state['next_node'] = routing_map.get(dominant_type, 'qa_guardrail')
    
    state = log_to_audit(state, "route_by_type", {"dominant_type": dominant_type, "routed_to": state['next_node']})
    
    return state


def node_techseo_agent(state: SEOAgentState) -> SEOAgentState:
    """
    TECH SEO SPECIALIST: Generate ChangeSet objects for on-page/technical fixes.
    Responsible for: schema, crawlability, structure, redirects, indexation.
    """
    print(f"[TECHSEO AGENT] Run {state['run_id']}: Generating technical fixes...")
    
    changesets = []
    for issue in state.get('issues', []):
        if issue.get('issue_type') == 'TECH_SEO':
            changeset = ChangeSet(
                url=issue['affected_urls'][0] if issue.get('affected_urls') else "example.com",
                change_type="schema_markup",
                before="<empty>",
                after='<script type="application/ld+json">{"@context": "https://schema.org", ...}</script>',
                rollback_plan="Remove added script tag",
                risk_level=IssueSeverity.LOW
            )
            changesets.append(changeset.to_dict())
    
    state['changesets'].extend(changesets)
    state['next_node'] = "qa_guardrail"
    state = log_to_audit(state, "techseo_agent", {"changesets_generated": len(changesets)})
    
    return state


def node_productseo_agent(state: SEOAgentState) -> SEOAgentState:
    """
    PRODUCT SEO SPECIALIST: Optimize product pages, CTR, snippets.
    Responsible for: product schema, meta tags, title/description optimization.
    """
    print(f"[PRODUCTSEO AGENT] Run {state['run_id']}: Generating product optimizations...")
    
    changesets = []
    for opp in state.get('opportunities', [])[:2]:
        changeset = ChangeSet(
            url="example.com/product",
            change_type="meta_tag",
            before="<meta name='description' content='Old description'>",
            after="<meta name='description' content='New optimized description with keyword'>",
            rollback_plan="Restore previous meta tag",
            risk_level=IssueSeverity.LOW
        )
        changesets.append(changeset.to_dict())
    
    state['changesets'].extend(changesets)
    state['next_node'] = "qa_guardrail"
    state = log_to_audit(state, "productseo_agent", {"changesets_generated": len(changesets)})
    
    return state


def node_content_agent(state: SEOAgentState) -> SEOAgentState:
    """
    CONTENT SPECIALIST: Recommend content improvements.
    Responsible for: keyword coverage, content gaps, E-E-A-T signals.
    """
    print(f"[CONTENT AGENT] Run {state['run_id']}: Recommending content improvements...")
    
    tickets = []
    for opp in state.get('opportunities', [])[:1]:
        ticket = Ticket(
            title=f"Content Review: {opp.get('title', 'Untitled')}",
            description=opp.get('description', 'Review and improve content'),
            assignee="content-team",
            priority=IssueSeverity.MEDIUM
        )
        tickets.append(ticket.to_dict())
    
    state['tickets'].extend(tickets)
    state['next_node'] = "qa_guardrail"
    state = log_to_audit(state, "content_agent", {"tickets_created": len(tickets)})
    
    return state


def node_authority_agent(state: SEOAgentState) -> SEOAgentState:
    """
    AUTHORITY SPECIALIST: Recommend link-building and authority strategies.
    Responsible for: backlink acquisition, authority growth, brand signals.
    """
    print(f"[AUTHORITY AGENT] Run {state['run_id']}: Recommending authority strategies...")
    
    tickets = []
    backlinks = state.get('inputs', {}).get('backlinks', {})
    if backlinks.get('toxic_links', 0) > 10:
        ticket = Ticket(
            title="Review and disavow toxic backlinks",
            description=f"{backlinks.get('toxic_links', 0)} toxic backlinks detected. Review and disavow.",
            assignee="link-team",
            priority=IssueSeverity.HIGH
        )
        tickets.append(ticket.to_dict())
    
    state['tickets'].extend(tickets)
    state['next_node'] = "qa_guardrail"
    state = log_to_audit(state, "authority_agent", {"tickets_created": len(tickets)})
    
    return state


print("✓ LangGraph nodes defined")

In [ ]:
# STEP 6: Define LangGraph Conditional Edges (Routing Logic)
# =============================================================================

def route_after_inputs(state: SEOAgentState) -> str:
    """
    After collecting inputs, always go to normalize.
    """
    return "normalize_inputs"


def route_after_collection(state: SEOAgentState) -> str:
    """
    After normalization, route based on findings.
    """
    if state.get('issues') or state.get('opportunities'):
        return "score_opportunities"
    else:
        return END


def route_after_specialist(state: SEOAgentState) -> str:
    """
    After a specialist agent, route to QA guardrail.
    """
    return "qa_guardrail"


def route_to_specialist(state: SEOAgentState) -> str:
    """
    Route to appropriate specialist based on dominant issue type.
    """
    issues = state.get('issues', [])
    if not issues:
        return "qa_guardrail"
    
    type_counts = {}
    for issue in issues:
        issue_type = issue.get('issue_type', 'UNKNOWN')
        type_counts[issue_type] = type_counts.get(issue_type, 0) + 1
    
    dominant_type = max(type_counts.items(), key=lambda x: x[1])[0] if type_counts else "UNKNOWN"
    
    routing_map = {
        'TECH_SEO': 'techseo_agent_node',
        'PRODUCT_SEO': 'productseo_agent_node',
        'CONTENT': 'content_agent_node',
        'AUTHORITY': 'authority_agent_node',
    }
    
    return routing_map.get(dominant_type, "qa_guardrail")


def node_qa_guardrail(state: SEOAgentState) -> SEOAgentState:
    """
    QA GUARDRAIL NODE: Validate all generated ChangeSet objects against safety rules.
    Rules:
      - ChangeSet must have valid 'before', 'after', and 'rollback_plan'
      - Risk level must be assessed
      - No write action without schema validation
    Output: approve OR flag for approval_interrupt_node
    """
    print(f"[QA GUARDRAIL] Run {state['run_id']}: Validating changesets...")
    
    changesets = state.get('changesets', [])
    requires_approval = False
    errors = []
    
    for cs in changesets:
        # Validate schema
        if not cs.get('before') or not cs.get('after'):
            errors.append(f"Changeset {cs['changeset_id']}: Missing before/after")
        if not cs.get('rollback_plan'):
            errors.append(f"Changeset {cs['changeset_id']}: No rollback plan")
        
        # Check risk level
        if cs.get('risk_level') in ['CRITICAL', 'HIGH']:
            requires_approval = True
    
    state['requires_approval'] = requires_approval
    if errors:
        state['errors'].extend(errors)
    
    state = log_to_audit(state, "qa_guardrail", {
        "changesets_validated": len(changesets),
        "errors": len(errors),
        "requires_approval": requires_approval
    })
    
    if requires_approval:
        state['next_node'] = "approval_interrupt"
    else:
        state['next_node'] = "execute_changeset"
    
    return state


def route_after_qa(state: SEOAgentState) -> str:
    """
    After QA, route to approval if high-risk, else execute.
    """
    if state.get('requires_approval'):
        return "approval_interrupt"
    else:
        return "execute_changeset"


def node_approval_interrupt(state: SEOAgentState) -> SEOAgentState:
    """
    APPROVAL INTERRUPT NODE (Human-in-the-Loop): Pause for manual approval.
    In LangGraph, this is an interrupt that halts execution until resumed.
    Output: approval_status -> APPROVED or REJECTED
    """
    print(f"[APPROVAL INTERRUPT] Run {state['run_id']}: Awaiting human approval...")
    
    print("\n" + "="*60)
    print("HUMAN-IN-THE-LOOP: Approval Required")
    print("="*60)
    print(f"Changesets to review: {len(state.get('changesets', []))}")
    for i, cs in enumerate(state.get('changesets', []), 1):
        print(f"\n  {i}. {cs.get('change_type')} @ {cs.get('url')}")
        print(f"     Risk: {cs.get('risk_level')}")
        print(f"     Before: {cs.get('before')[:50]}...")
        print(f"     After:  {cs.get('after')[:50]}...")
    
    # MOCK: Auto-approve for demo
    # In production, this would be a real approval workflow
    state['approval_status'] = 'APPROVED'
    state['approval_notes'] = 'Auto-approved for demo (would be manual in production)'
    
    state = log_to_audit(state, "approval_interrupt", {"status": state['approval_status']})
    state['next_node'] = "execute_changeset"
    
    return state


def node_execute_changeset(state: SEOAgentState) -> SEOAgentState:
    """
    EXECUTE NODE: Apply approved ChangeSet objects to live systems.
    Only executes if:
      - approval_status == APPROVED
      - rollback_plan is present
      - changeset.valid == true
    """
    print(f"[EXECUTE] Run {state['run_id']}: Executing approved changesets...")
    
    if state.get('approval_status') != 'APPROVED':
        state['execution_errors'].append("Changesets not approved")
        state['next_node'] = "persist_run"
        return state
    
    executed = []
    for cs in state.get('changesets', []):
        # MOCK: Simulate execution
        cs['status'] = 'COMPLETED'
        executed.append(cs)
        print(f"  ✓ Executed: {cs.get('change_type')} @ {cs.get('url')}")
    
    state['executed_changesets'] = executed
    state = log_to_audit(state, "execute_changeset", {"executed": len(executed)})
    state['next_node'] = "validate_metrics"
    
    return state


def node_validate_metrics(state: SEOAgentState) -> SEOAgentState:
    """
    VALIDATE NODE: Compare before/after metrics.
    Collects post-execution metrics and validates improvement.
    """
    print(f"[VALIDATE] Run {state['run_id']}: Collecting post-execution metrics...")
    
    # MOCK: Simulate post-execution metrics
    state['metrics_before'] = {
        'indexed_pages': 125,
        'avg_ctr': 0.065,
        'avg_position': 4.5,
        'organic_traffic': 2000
    }
    
    state['metrics_after'] = {
        'indexed_pages': 135,  # +10
        'avg_ctr': 0.078,      # +20%
        'avg_position': 3.8,   # Improved
        'organic_traffic': 2400  # +20%
    }
    
    state['validation'] = {
        'passed': True,
        'improvements': {
            'indexed_pages': 10,
            'avg_ctr_improvement_pct': 20,
            'position_improvement': 0.7,
            'traffic_improvement_pct': 20
        }
    }
    
    state = log_to_audit(state, "validate_metrics", {"validation_passed": True})
    state['next_node'] = "persist_run"
    
    return state


def node_persist_run(state: SEOAgentState) -> SEOAgentState:
    """
    PERSIST NODE: Store complete run state for audit, replay, and learning.
    Persists to: audit_log, metrics, changesets, tickets for future reference.
    """
    print(f"[PERSIST] Run {state['run_id']}: Archiving run state...")
    
    # Add final entry to audit log
    state = log_to_audit(state, "persist_run", {
        "total_changesets": len(state.get('executed_changesets', [])),
        "total_tickets": len(state.get('tickets', [])),
        "total_issues": len(state.get('issues', []))
    })
    
    print(f"\n  Run ID: {state['run_id']}")
    print(f"  Status: COMPLETED")
    print(f"  Issues Found: {len(state.get('issues', []))}")
    print(f"  Opportunities: {len(state.get('opportunities', []))}")
    print(f"  Changesets Executed: {len(state.get('executed_changesets', []))}")
    print(f"  Tickets Created: {len(state.get('tickets', []))}")
    print(f"  Audit Log Entries: {len(state.get('audit_log', []))}")
    
    state['current_route'] = 'COMPLETED'
    
    return state


print("✓ Conditional edges and remaining nodes defined")

In [ ]:
# STEP 7: Build LangGraph StateGraph (Main Graph)
# =============================================================================

def build_langgraph() -> CompiledGraph:
    """
    Construct the LangGraph StateGraph with all nodes and edges.
    
    Graph flow:
      START → collect_inputs → normalize → score → route_specialist
             → [techseo|productseo|content|authority]_agent
             → qa_guardrail → [approval_interrupt|execute] → validate → persist → END
    """
    
    # Initialize graph
    graph = StateGraph(SEOAgentState)
    
    # Add nodes
    graph.add_node("collect_inputs", node_collect_inputs)
    graph.add_node("normalize_inputs", node_normalize_inputs)
    graph.add_node("score_opportunities", node_score_opportunities)
    graph.add_node("route_by_type", node_route_by_type)
    
    # Specialist agents
    graph.add_node("techseo_agent_node", node_techseo_agent)
    graph.add_node("productseo_agent_node", node_productseo_agent)
    graph.add_node("content_agent_node", node_content_agent)
    graph.add_node("authority_agent_node", node_authority_agent)
    
    # Guardrails & execution
    graph.add_node("qa_guardrail", node_qa_guardrail)
    graph.add_node("approval_interrupt", node_approval_interrupt)
    graph.add_node("execute_changeset", node_execute_changeset)
    graph.add_node("validate_metrics", node_validate_metrics)
    graph.add_node("persist_run", node_persist_run)
    
    # Add edges (direct path)
    graph.add_edge("START", "collect_inputs")
    graph.add_edge("collect_inputs", "normalize_inputs")
    graph.add_edge("normalize_inputs", "score_opportunities")
    graph.add_edge("score_opportunities", "route_by_type")
    
    # Add conditional edges (routing to specialists)
    graph.add_conditional_edges(
        "route_by_type",
        route_to_specialist,
        {
            "techseo_agent_node": "techseo_agent_node",
            "productseo_agent_node": "productseo_agent_node",
            "content_agent_node": "content_agent_node",
            "authority_agent_node": "authority_agent_node",
            "qa_guardrail": "qa_guardrail"
        }
    )
    
    # All specialists → QA
    graph.add_edge("techseo_agent_node", "qa_guardrail")
    graph.add_edge("productseo_agent_node", "qa_guardrail")
    graph.add_edge("content_agent_node", "qa_guardrail")
    graph.add_edge("authority_agent_node", "qa_guardrail")
    
    # Conditional edge: QA → Approval or Execute
    graph.add_conditional_edges(
        "qa_guardrail",
        route_after_qa,
        {
            "approval_interrupt": "approval_interrupt",
            "execute_changeset": "execute_changeset"
        }
    )
    
    # Approval → Execute
    graph.add_edge("approval_interrupt", "execute_changeset")
    
    # Execution flow
    graph.add_edge("execute_changeset", "validate_metrics")
    graph.add_edge("validate_metrics", "persist_run")
    graph.add_edge("persist_run", END)
    
    # Compile graph
    compiled_graph = graph.compile()
    
    return compiled_graph


# Build the graph
langgraph_instance = build_langgraph()

print("✓ LangGraph StateGraph compiled successfully")
print("\nLangGraph Nodes:")
print(f"  - Total nodes: {len(langgraph_instance.nodes)}")
print(f"  - Entry point: START")
print(f"  - Exit point: END")

In [ ]:
# STEP 8: CrewAI Integration (Framework Adapter)
# =============================================================================
# CrewAI expresses the same graph as Agents + Tasks + Flows
# This adapter allows the universal graph to work with CrewAI

from typing import List, Optional
from dataclasses import dataclass


@dataclass
class CrewAITask:
    """Simplified CrewAI Task representation."""
    id: str
    name: str
    description: str
    agent_id: str
    depends_on: List[str] = None  # Task IDs this depends on
    
    def __init__(self, id: str, name: str, description: str, agent_id: str, depends_on=None):
        self.id = id
        self.name = name
        self.description = description
        self.agent_id = agent_id
        self.depends_on = depends_on or []


@dataclass
class CrewAIAgent:
    """Simplified CrewAI Agent representation."""
    id: str
    name: str
    role: str
    description: str
    tools: List[str] = None
    
    def __init__(self, id: str, name: str, role: str, description: str, tools=None):
        self.id = id
        self.name = name
        self.role = role
        self.description = description
        self.tools = tools or []


class UniversalSEOFlow:
    """
    CrewAI Flow-based adapter for the universal SEO agent graph.
    Implements the same workflow as LangGraph but using CrewAI primitives.
    """
    
    def __init__(self):
        self.state = None
        self.agents = {}
        self.tasks = {}
        self.flow_graph = {}
        self._init_agents()
        self._init_tasks()
        self._build_flow()
    
    def _init_agents(self):
        """Initialize specialist agents."""
        self.agents = {
            "data_collector": CrewAIAgent(
                id="agent_collector",
                name="Data Collector",
                role="SEO Data Aggregator",
                description="Collects data from GSC, GA4, crawlers, SERP, backlinks",
                tools=["collect_gsc", "collect_ga4", "crawl_website", "collect_serp", "collect_backlinks"]
            ),
            "analyst": CrewAIAgent(
                id="agent_analyst",
                name="SEO Analyst",
                role="Issue & Opportunity Analyst",
                description="Normalizes, detects anomalies, and scores opportunities",
                tools=["normalize_inputs"]
            ),
            "techseo": CrewAIAgent(
                id="agent_techseo",
                name="TechSEO Specialist",
                role="Technical SEO Expert",
                description="Handles indexation, schema, crawlability, structure issues",
                tools=[]
            ),
            "productseo": CrewAIAgent(
                id="agent_productseo",
                name="Product SEO Specialist",
                role="Product Optimization Expert",
                description="Optimizes product pages, CTR, meta tags",
                tools=[]
            ),
            "content": CrewAIAgent(
                id="agent_content",
                name="Content Strategist",
                role="Content SEO Expert",
                description="Recommends content improvements and E-E-A-T enhancements",
                tools=[]
            ),
            "authority": CrewAIAgent(
                id="agent_authority",
                name="Authority Builder",
                role="Link & Authority Expert",
                description="Develops link-building and authority growth strategies",
                tools=[]
            ),
            "qa": CrewAIAgent(
                id="agent_qa",
                name="QA Validator",
                role="Quality & Safety Guardrail",
                description="Validates changesets, enforces SEO safety rules",
                tools=[]
            )
        }
    
    def _init_tasks(self):
        """Initialize CrewAI tasks (equivalent to LangGraph nodes)."""
        self.tasks = {
            "collect_task": CrewAITask(
                id="task_collect",
                name="Collect SEO Data",
                description="Gather snapshots from GSC, GA4, crawl, SERP, backlinks",
                agent_id="agent_collector"
            ),
            "analyze_task": CrewAITask(
                id="task_analyze",
                name="Analyze & Normalize",
                description="Normalize inputs and detect issues/opportunities",
                agent_id="agent_analyst",
                depends_on=["task_collect"]
            ),
            "techseo_task": CrewAITask(
                id="task_techseo",
                name="Generate Tech Fixes",
                description="Create ChangeSet objects for technical SEO issues",
                agent_id="agent_techseo",
                depends_on=["task_analyze"]
            ),
            "productseo_task": CrewAITask(
                id="task_productseo",
                name="Optimize Products",
                description="Generate optimizations for product pages",
                agent_id="agent_productseo",
                depends_on=["task_analyze"]
            ),
            "content_task": CrewAITask(
                id="task_content",
                name="Recommend Content",
                description="Create tickets for content improvements",
                agent_id="agent_content",
                depends_on=["task_analyze"]
            ),
            "authority_task": CrewAITask(
                id="task_authority",
                name="Authority Strategy",
                description="Develop link-building and authority recommendations",
                agent_id="agent_authority",
                depends_on=["task_analyze"]
            ),
            "qa_task": CrewAITask(
                id="task_qa",
                name="QA Validation",
                description="Validate all changesets and enforce safety rules",
                agent_id="agent_qa",
                depends_on=["task_techseo", "task_productseo", "task_content", "task_authority"]
            ),
            "execute_task": CrewAITask(
                id="task_execute",
                name="Execute Changes",
                description="Apply approved changesets to live systems",
                agent_id="agent_collector",  # Could be its own executor
                depends_on=["task_qa"]
            ),
            "validate_task": CrewAITask(
                id="task_validate",
                name="Validate Metrics",
                description="Compare before/after metrics and validate improvements",
                agent_id="agent_analyst",
                depends_on=["task_execute"]
            ),
            "persist_task": CrewAITask(
                id="task_persist",
                name="Persist Run State",
                description="Archive complete run for audit and replay",
                agent_id="agent_analyst",
                depends_on=["task_validate"]
            )
        }
    
    def _build_flow(self):
        """Build flow graph topology (edges between tasks)."""
        self.flow_graph = {
            "task_collect": ["task_analyze"],
            "task_analyze": ["task_techseo", "task_productseo", "task_content", "task_authority"],
            "task_techseo": ["task_qa"],
            "task_productseo": ["task_qa"],
            "task_content": ["task_qa"],
            "task_authority": ["task_qa"],
            "task_qa": ["task_execute"],
            "task_execute": ["task_validate"],
            "task_validate": ["task_persist"],
            "task_persist": []  # END
        }
    
    def execute_flow(self, initial_state: Optional[SEOAgentState] = None) -> SEOAgentState:
        """
        Execute the CrewAI flow (in-memory, simulated).
        In production, this would dispatch to actual CrewAI Crew/Flow.
        """
        print("="*70)
        print("CREWAI FLOW: Universal SEO Agent")
        print("="*70)
        
        self.state = initial_state or initialize_state()
        executed_tasks = set()
        pending_tasks = set(self.tasks.keys())
        
        iteration = 0
        max_iterations = 20  # Prevent infinite loops
        
        while pending_tasks and iteration < max_iterations:
            iteration += 1
            print(f"\n[CrewAI Iteration {iteration}]")
            
            # Find executable tasks (dependencies met)
            executable = []
            for task_id in pending_tasks:
                task = self.tasks[task_id]
                if all(dep in executed_tasks for dep in task.depends_on):
                    executable.append(task_id)
            
            if not executable:
                if pending_tasks:
                    print(f"WARNING: Deadlock detected. Pending: {pending_tasks}")
                break
            
            # Execute tasks in this iteration
            for task_id in executable:
                task = self.tasks[task_id]
                agent = self.agents[task.agent_id]
                
                print(f"  ► Task: {task.name}")
                print(f"    Agent: {agent.name}")
                
                # Execute corresponding LangGraph node
                node_map = {
                    "task_collect": node_collect_inputs,
                    "task_analyze": node_normalize_inputs,
                    "task_techseo": node_techseo_agent,
                    "task_productseo": node_productseo_agent,
                    "task_content": node_content_agent,
                    "task_authority": node_authority_agent,
                    "task_qa": node_qa_guardrail,
                    "task_execute": node_execute_changeset,
                    "task_validate": node_validate_metrics,
                    "task_persist": node_persist_run
                }
                
                if task_id in node_map:
                    self.state = node_map[task_id](self.state)
                
                executed_tasks.add(task_id)
                pending_tasks.remove(task_id)
                print(f"    ✓ Completed")
        
        print(f"\nFlow completed in {iteration} iterations")
        return self.state


print("✓ CrewAI adapter (UniversalSEOFlow) implemented")

In [ ]:
# STEP 9: Unified Framework Interface (Framework Agnostic)
# =============================================================================

class UniversalSEOAgent:
    """
    Universal wrapper that works with both LangGraph and CrewAI.
    Provides a single entry point for orchestrating SEO agent workflows.
    """
    
    def __init__(self, framework: str = "langgraph"):
        """
        Initialize the universal agent.
        
        Args:
            framework: "langgraph" or "crewai"
        """
        self.framework = framework.lower()
        self.state = None
        self.graph = None
        
        if self.framework == "langgraph":
            self.graph = langgraph_instance
            print(f"✓ Initialized with LangGraph")
        elif self.framework == "crewai":
            self.graph = UniversalSEOFlow()
            print(f"✓ Initialized with CrewAI")
        else:
            raise ValueError(f"Unsupported framework: {framework}")
    
    def run(self, domain: str = "example.com", initial_state: Optional[SEOAgentState] = None) -> SEOAgentState:
        """
        Execute the SEO agent workflow.
        
        Args:
            domain: Domain to analyze
            initial_state: Optional initial state (default: fresh state)
        
        Returns:
            Final state after workflow completion
        """
        
        if initial_state is None:
            initial_state = initialize_state(run_id=f"seo-run-{uuid.uuid4().hex[:8]}")
        
        print(f"\n{'='*70}")
        print(f"UNIVERSAL SEO AGENT - Framework: {self.framework.upper()}")
        print(f"{'='*70}")
        print(f"Domain: {domain}")
        print(f"Run ID: {initial_state['run_id']}")
        print(f"{'='*70}\n")
        
        if self.framework == "langgraph":
            return self._run_langgraph(initial_state)
        elif self.framework == "crewai":
            return self._run_crewai(initial_state)
    
    def _run_langgraph(self, initial_state: SEOAgentState) -> SEOAgentState:
        """Execute workflow using LangGraph."""
        print("Framework: LangGraph (Deterministic State Machine)\n")
        
        final_state = self.graph.invoke(initial_state)
        return final_state
    
    def _run_crewai(self, initial_state: SEOAgentState) -> SEOAgentState:
        """Execute workflow using CrewAI."""
        print("Framework: CrewAI (Agent Task-based Orchestration)\n")
        
        final_state = self.graph.execute_flow(initial_state)
        return final_state
    
    def get_summary(self, state: SEOAgentState) -> Dict[str, Any]:
        """Generate a summary of the workflow execution."""
        return {
            'run_id': state.get('run_id'),
            'created_at': state.get('created_at'),
            'status': 'COMPLETED',
            'total_issues_found': len(state.get('issues', [])),
            'total_opportunities': len(state.get('opportunities', [])),
            'changesets_created': len(state.get('changesets', [])),
            'changesets_executed': len(state.get('executed_changesets', [])),
            'tickets_created': len(state.get('tickets', [])),
            'audit_log_entries': len(state.get('audit_log', [])),
            'metrics_improvement': {
                'indexed_pages': state.get('metrics_after', {}).get('indexed_pages', 0) - state.get('metrics_before', {}).get('indexed_pages', 0),
                'ctr_improvement_pct': ((state.get('metrics_after', {}).get('avg_ctr', 0) - state.get('metrics_before', {}).get('avg_ctr', 0)) / max(state.get('metrics_before', {}).get('avg_ctr', 1), 0.001)) * 100,
                'traffic_improvement_pct': ((state.get('metrics_after', {}).get('organic_traffic', 0) - state.get('metrics_before', {}).get('organic_traffic', 0)) / max(state.get('metrics_before', {}).get('organic_traffic', 1), 0.001)) * 100
            }
        }


print("✓ UniversalSEOAgent framework adapter implemented")

In [ ]:
# STEP 10: Graph Visualization & Validation
# =============================================================================

def visualize_graph_structure() -> Dict[str, Any]:
    """
    Generate a visualization-ready representation of the universal graph.
    Compatible with graph visualization tools (Graphviz, Cytoscape, etc).
    """
    
    graph_structure = {
        "name": "Universal SEO Agent Graph",
        "description": "LangGraph + CrewAI powered by LangChain tools",
        "nodes": {
            "START": {"type": "entry", "label": "Start"},
            "collect_inputs": {"type": "action", "label": "Collect Data", "tools": ["GSC", "GA4", "Crawler", "SERP", "Backlinks"]},
            "normalize_inputs": {"type": "action", "label": "Normalize & Detect"},
            "score_opportunities": {"type": "action", "label": "Score (Impact×Conf/Effort)"},
            "route_by_type": {"type": "router", "label": "Route by Type"},
            "techseo_agent_node": {"type": "specialist", "label": "TechSEO Agent", "handles": ["indexation", "schema", "crawlability"]},
            "productseo_agent_node": {"type": "specialist", "label": "ProductSEO Agent", "handles": ["product_pages", "meta", "ctr"]},
            "content_agent_node": {"type": "specialist", "label": "Content Agent", "handles": ["content_gaps", "e-e-a-t"]},
            "authority_agent_node": {"type": "specialist", "label": "Authority Agent", "handles": ["backlinks", "brand_signals"]},
            "qa_guardrail": {"type": "guardrail", "label": "QA Validation", "rules": ["schema_valid", "rollback_present", "risk_assessed"]},
            "approval_interrupt": {"type": "hitl", "label": "Human Approval", "mode": "interrupt"},
            "execute_changeset": {"type": "action", "label": "Execute Changes"},
            "validate_metrics": {"type": "action", "label": "Validate Metrics"},
            "persist_run": {"type": "action", "label": "Persist & Archive"},
            "END": {"type": "exit", "label": "End"}
        },
        "edges": {
            "START": ["collect_inputs"],
            "collect_inputs": ["normalize_inputs"],
            "normalize_inputs": ["score_opportunities"],
            "score_opportunities": ["route_by_type"],
            "route_by_type": ["techseo_agent_node", "productseo_agent_node", "content_agent_node", "authority_agent_node", "qa_guardrail"],
            "techseo_agent_node": ["qa_guardrail"],
            "productseo_agent_node": ["qa_guardrail"],
            "content_agent_node": ["qa_guardrail"],
            "authority_agent_node": ["qa_guardrail"],
            "qa_guardrail": ["approval_interrupt", "execute_changeset"],
            "approval_interrupt": ["execute_changeset"],
            "execute_changeset": ["validate_metrics"],
            "validate_metrics": ["persist_run"],
            "persist_run": ["END"]
        },
        "conditional_routing": {
            "route_by_type": {
                "condition": "dominant_issue_type",
                "routes": {
                    "TECH_SEO": "techseo_agent_node",
                    "PRODUCT_SEO": "productseo_agent_node",
                    "CONTENT": "content_agent_node",
                    "AUTHORITY": "authority_agent_node",
                    "UNKNOWN": "qa_guardrail"
                }
            },
            "qa_guardrail": {
                "condition": "requires_approval",
                "routes": {
                    "True": "approval_interrupt",
                    "False": "execute_changeset"
                }
            }
        },
        "state_flow": {
            "initial": {"run_id": "UUID", "inputs": "empty", "findings": "empty"},
            "after_collect": {"inputs": {"gsc": {}, "ga4": {}, "crawl": {}, "serp": {}, "backlinks": {}}},
            "after_normalize": {"issues": "[Issue]", "opportunities": "[Opportunity]"},
            "after_score": {"scores": "{}"},
            "after_specialist": {"changesets": "[ChangeSet]", "tickets": "[Ticket]"},
            "after_qa": {"requires_approval": "bool"},
            "after_execute": {"executed_changesets": "[ChangeSet]"},
            "after_validate": {"metrics_before": "{}", "metrics_after": "{}", "validation": "{}"},
            "final": {"audit_log": "[LogEntry]", "status": "COMPLETED"}
        }
    }
    
    return graph_structure


def print_graph_ascii() -> str:
    """Print ASCII representation of the graph flow."""
    
    ascii_graph = r"""
╔════════════════════════════════════════════════════════════════════════════╗
║                    UNIVERSAL SEO AGENT GRAPH TOPOLOGY                      ║
║                    (LangGraph + CrewAI + LangChain)                        ║
╚════════════════════════════════════════════════════════════════════════════╝

                                   START
                                     │
                                     ▼
                          ┌─────────────────────┐
                          │  COLLECT_INPUTS     │  ◄── LangChain Tools:
                          │                     │      • GSC, GA4, Crawler
                          │  Tools:             │      • SERP, Backlinks
                          │  • collect_gsc      │
                          │  • collect_ga4      │
                          │  • crawl_website    │
                          │  • collect_serp     │
                          │  • collect_backlinks│
                          └─────────────────────┘
                                     │
                                     ▼
                          ┌─────────────────────┐
                          │ NORMALIZE_INPUTS    │
                          │ (Issue + Oppty)     │
                          └─────────────────────┘
                                     │
                                     ▼
                          ┌─────────────────────┐
                          │ SCORE_OPPORTUNITIES │  ◄── Impact × Confidence
                          │ (Prioritize)        │      ÷ Effort
                          └─────────────────────┘
                                     │
                                     ▼
                          ┌─────────────────────┐
                          │   ROUTE_BY_TYPE    │  ◄── Conditional Edge
                          │ (Dominant Issue)    │      (IssueType → Agent)
                          └─────────────────────┘
                                     │
              ┌──────────────────────┼──────────────────────┐
              │                      │                      │
        TECH_SEO                  PRODUCT                CONTENT
              │                      │                      │
              ▼                      ▼                      ▼
    ┌─────────────────┐   ┌─────────────────┐  ┌─────────────────┐
    │ TECHSEO_AGENT   │   │PRODUCTSEO_AGENT │  │ CONTENT_AGENT   │
    │ (ChangeSet)     │   │ (ChangeSet)     │  │ (Ticket)        │
    └─────────────────┘   └─────────────────┘  └─────────────────┘
              │                      │                      │
              │                      └──────────┬───────────┘
              │                                 │
              └─────────────────────┬───────────┘
                                    │
                           AUTHORITY_AGENT
                           (Ticket)
                                    │
                                    ▼
                          ┌─────────────────────┐
                          │  QA_GUARDRAIL       │  ◄── Validation Rules:
                          │ (Validate ChangeSet │      • Schema valid
                          │  + Risk Assess)     │      • Rollback present
                          └─────────────────────┘      • Risk assessed
                                    │
                    ┌───────────────┼───────────────┐
                    │               │               │
              risk < 4        risk >= 4       (No issues)
                    │               │               │
                    ▼               ▼               │
          ┌──────────────────┐    ┌──────────────┐ │
          │ EXECUTE_CHANGESET│◄───│  APPROVAL    │ │
          │ (Low Risk)       │    │ INTERRUPT    │ │
          │                  │    │ (HITL)       │ │
          └──────────────────┘    └──────────────┘ │
                    │                               │
                    └───────────────┬───────────────┘
                                    │
                                    ▼
                          ┌─────────────────────┐
                          │ VALIDATE_METRICS    │  ◄── Before/After:
                          │ (Measure Impact)    │      • Indexed pages
                          └─────────────────────┘      • CTR, Position
                                    │                   • Traffic
                                    ▼
                          ┌─────────────────────┐
                          │  PERSIST_RUN        │  ◄── Audit Log:
                          │ (Archive + Audit)   │      • State snapshot
                          └─────────────────────┘      • All decisions
                                    │                   • Replay capability
                                    ▼
                                   END

╔════════════════════════════════════════════════════════════════════════════╗
║  KEY FEATURES:                                                             ║
║  ✓ Deterministic state machine (LangGraph)                                │
║  ✓ Framework-agnostic (works with CrewAI too)                            │
║  ✓ Conditional routing (specialist agents by issue type)                 │
║  ✓ Human-in-the-loop (approval interrupts for high-risk actions)         │
║  ✓ Complete audit trail (timestamps, decisions, errors)                  │
║  ✓ Structured outputs (Issue, Opportunity, ChangeSet, Ticket)           │
║  ✓ Rollback plans (every change is reversible)                          │
║  ✓ Metrics tracking (before/after validation)                           │
╚════════════════════════════════════════════════════════════════════════════╝
"""
    
    return ascii_graph


# Generate and display
graph_structure = visualize_graph_structure()

print(print_graph_ascii())
print("\n✓ Graph visualization complete")

In [ ]:
# STEP 11: Test & Validate - LangGraph Execution
# =============================================================================

print("\n" + "="*80)
print("EXECUTION TEST 1: LangGraph Framework")
print("="*80 + "\n")

# Initialize agent with LangGraph
agent_langgraph = UniversalSEOAgent(framework="langgraph")

# Create initial state
initial_state = initialize_state()

# Execute workflow
try:
    final_state_langgraph = agent_langgraph.run(
        domain="example.com",
        initial_state=initial_state
    )
    
    # Display results
    summary = agent_langgraph.get_summary(final_state_langgraph)
    
    print("\n" + "="*80)
    print("LANGGRAPH EXECUTION SUMMARY")
    print("="*80)
    print(json.dumps(summary, indent=2))
    
    print("\nFinal State Audit Log (last 5 entries):")
    for entry in final_state_langgraph.get('audit_log', [])[-5:]:
        print(f"  • {entry['timestamp']}: {entry['action']}")
    
    print("\n✓ LangGraph execution completed successfully")
    
except Exception as e:
    print(f"✗ Error during LangGraph execution: {str(e)}")
    import traceback
    traceback.print_exc()

In [ ]:
# STEP 12: Test & Validate - CrewAI Execution
# =============================================================================

print("\n" + "="*80)
print("EXECUTION TEST 2: CrewAI Framework")
print("="*80 + "\n")

# Initialize agent with CrewAI
agent_crewai = UniversalSEOAgent(framework="crewai")

# Create initial state (fresh for CrewAI test)
initial_state_crewai = initialize_state()

# Execute workflow
try:
    final_state_crewai = agent_crewai.run(
        domain="example.com",
        initial_state=initial_state_crewai
    )
    
    # Display results
    summary_crewai = agent_crewai.get_summary(final_state_crewai)
    
    print("\n" + "="*80)
    print("CREWAI EXECUTION SUMMARY")
    print("="*80)
    print(json.dumps(summary_crewai, indent=2))
    
    print("\nFinal State Audit Log (last 5 entries):")
    for entry in final_state_crewai.get('audit_log', [])[-5:]:
        print(f"  • {entry['timestamp']}: {entry['action']}")
    
    print("\n✓ CrewAI execution completed successfully")
    
except Exception as e:
    print(f"✗ Error during CrewAI execution: {str(e)}")
    import traceback
    traceback.print_exc()

In [ ]:
# STEP 13: Comparative Analysis & Architecture Documentation
# =============================================================================

def compare_frameworks(langgraph_state, crewai_state):
    """
    Compare LangGraph and CrewAI execution outcomes.
    Both should produce equivalent results (same graph, different execution models).
    """
    
    comparison = {
        "execution_model": {
            "langgraph": "State Machine (deterministic, linear state flow)",
            "crewai": "Task-based (agent delegation, parallel-capable tasks)"
        },
        "run_metrics": {
            "langgraph": {
                "run_id": langgraph_state.get('run_id'),
                "total_audit_entries": len(langgraph_state.get('audit_log', [])),
                "execution_status": langgraph_state.get('current_route')
            },
            "crewai": {
                "run_id": crewai_state.get('run_id'),
                "total_audit_entries": len(crewai_state.get('audit_log', [])),
                "execution_status": crewai_state.get('current_route')
            }
        },
        "outputs_comparison": {
            "issues_found": {
                "langgraph": len(langgraph_state.get('issues', [])),
                "crewai": len(crewai_state.get('issues', []))
            },
            "opportunities_identified": {
                "langgraph": len(langgraph_state.get('opportunities', [])),
                "crewai": len(crewai_state.get('opportunities', []))
            },
            "changesets_generated": {
                "langgraph": len(langgraph_state.get('changesets', [])),
                "crewai": len(crewai_state.get('changesets', []))
            },
            "tickets_created": {
                "langgraph": len(langgraph_state.get('tickets', [])),
                "crewai": len(crewai_state.get('tickets', []))
            },
            "changesets_executed": {
                "langgraph": len(langgraph_state.get('executed_changesets', [])),
                "crewai": len(crewai_state.get('executed_changesets', []))
            }
        },
        "framework_characteristics": {
            "langgraph": {
                "pros": [
                    "Perfect for deterministic workflows",
                    "Built-in persistence & checkpointing",
                    "Interrupts for human-in-the-loop",
                    "Excellent for graph visualization",
                    "Strong streaming & partial output support"
                ],
                "cons": [
                    "Less flexibility for parallel tasks",
                    "Requires explicit graph definition"
                ]
            },
            "crewai": {
                "pros": [
                    "Natural agent-based delegation",
                    "Better for multi-agent collaboration",
                    "Hierarchical process support",
                    "Event-driven workflows",
                    "Task dependencies & memory integration"
                ],
                "cons": [
                    "Less deterministic",
                    "Harder to debug/trace complex flows"
                ]
            }
        }
    }
    
    return comparison


print("\n" + "="*80)
print("FRAMEWORK COMPARISON: LangGraph vs CrewAI")
print("="*80 + "\n")

# Perform comparison
comparison_results = compare_frameworks(final_state_langgraph, final_state_crewai)

print("1. EXECUTION MODELS")
print("-" * 80)
print(f"LangGraph:  {comparison_results['execution_model']['langgraph']}")
print(f"CrewAI:     {comparison_results['execution_model']['crewai']}")

print("\n2. OUTPUTS COMPARISON")
print("-" * 80)
for metric, values in comparison_results['outputs_comparison'].items():
    lg_val = values['langgraph']
    ca_val = values['crewai']
    match = "✓ MATCH" if lg_val == ca_val else f"✗ DIFFER ({lg_val} vs {ca_val})"
    print(f"{metric:30} LangGraph: {lg_val:3} | CrewAI: {ca_val:3} | {match}")

print("\n3. FRAMEWORK STRENGTHS")
print("-" * 80)
print("LangGraph Pros:")
for pro in comparison_results['framework_characteristics']['langgraph']['pros']:
    print(f"  ✓ {pro}")

print("\nCrewAI Pros:")
for pro in comparison_results['framework_characteristics']['crewai']['pros']:
    print(f"  ✓ {pro}")

print("\n✓ Framework comparison complete")

In [ ]:
# STEP 14: Architecture Reference & Best Practices
# =============================================================================

architecture_doc = """
╔════════════════════════════════════════════════════════════════════════════╗
║         UNIVERSAL SEO AGENT: Architecture & Implementation Guide           ║
║                   (LangGraph + CrewAI + LangChain)                         ║
╚════════════════════════════════════════════════════════════════════════════╝

## PART 1: CORE ARCHITECTURE

### 1.1 State Schema (SEOAgentState)
The unified state object is the "single source of truth" for any run:

├─ Metadata (run_id, created_at)
├─ Inputs (GSC, GA4, crawl, SERP, backlinks snapshots)
├─ Findings (normalized issues + opportunities)
├─ Scores (Impact × Confidence ÷ Effort prioritization)
├─ Plan (chosen actions: ChangeSet + Ticket objects)
├─ Approvals (status + reviewer notes for HITL gates)
├─ Execution (what was applied + where)
├─ Validation (before/after metrics + improvement checks)
└─ Audit Log (complete trace of all decisions, timestamps, errors)

### 1.2 Structured Output Types (Prevent Model Hallucination)
✓ Issue (title, description, type, severity, affected_urls, source)
✓ Opportunity (title, description, type, impact, confidence, effort, impact_score)
✓ ChangeSet (url, change_type, before, after, rollback_plan, risk_level, status)
✓ Ticket (title, description, assignee, priority, related_issues, status)

### 1.3 LangChain Tools (Reality Connectors)
Tools are the agent's only way to touch the SEO world:
✓ collect_gsc_data() → GSC performance metrics
✓ collect_ga4_data() → User engagement + conversions
✓ crawl_website() → On-page issues + structure problems
✓ collect_serp_snapshot() → Current ranking positions + snippets
✓ collect_backlinks() → Authority + link health metrics
✓ normalize_inputs() → Consolidate into unified schema

### 1.4 Graph Nodes (Processing Stages)

OBSERVE STAGE:
  collect_inputs → [GSC, GA4, crawl, SERP, backlinks snapshots]
  
DIAGNOSE STAGE:
  normalize_inputs → [Issue, Opportunity objects]
  score_opportunities → [rank by Impact×Conf/Effort]
  route_by_type → [Conditional: send to appropriate specialist]

RECOMMEND STAGE:
  techseo_agent_node → [ChangeSet for on-page, schema, structure]
  productseo_agent_node → [ChangeSet for product pages, CTR]
  content_agent_node → [Ticket for content improvements]
  authority_agent_node → [Ticket for link-building strategy]

GUARD STAGE:
  qa_guardrail → [Validate ChangeSet schema, risk check]
  approval_interrupt → [HITL pause for high-risk changes]
  
EXECUTE STAGE:
  execute_changeset → [Apply only approved, low-risk changes]
  
VALIDATE STAGE:
  validate_metrics → [Before/after comparison, improvement check]
  
PERSIST STAGE:
  persist_run → [Archive state for audit trail + replay capability]

### 1.5 Conditional Edges (Intelligent Routing)

route_by_type:
  dominat_issue_type == "TECH_SEO" → techseo_agent_node
  dominant_issue_type == "PRODUCT_SEO" → productseo_agent_node
  dominant_issue_type == "CONTENT" → content_agent_node
  dominant_issue_type == "AUTHORITY" → authority_agent_node
  else → qa_guardrail

qa_guardrail:
  requires_approval == True → approval_interrupt
  requires_approval == False → execute_changeset

---

## PART 2: FRAMEWORK IMPLEMENTATIONS

### 2.1 LangGraph Implementation

The LangGraph StateGraph is a state machine:
  • Nodes = functions that read state → do work → return updated state
  • Edges = deterministic transitions or conditional routing
  • State = immutable snapshots carried through the pipeline
  • Persistence = checkpointers + threads for resume/replay
  • Interrupts = pause points for human approval

KEY LangGraph FEATURES:
  ✓ Deterministic (same input → same output)
  ✓ Observable (trace all state changes)
  ✓ Resumable (checkpointer stores state, can resume from any node)
  ✓ HITL (interrupts pause for approvals)
  ✓ Streaming (can stream outputs as they're generated)

BEST FOR: Deterministic workflows, audit trails, replay capability

### 2.2 CrewAI Implementation

The CrewAI Flow is an agent-task network:
  • Agents = specialists with specific roles/tools
  • Tasks = units of work assigned to agents
  • Flows = event-driven coordination (start/listen/route)
  • Dependencies = task.depends_on specifies execution order
  • Memory = short/long/entity memory for agent context

KEY CrewAI FEATURES:
  ✓ Agent-based (natural delegation to specialists)
  ✓ Flexible (tasks can be parallel if deps allow)
  ✓ Memory-aware (agents remember context across tasks)
  ✓ Hierarchical (manager agent can oversee workers)
  ✓ Event-driven (listeners hook into task lifecycle)

BEST FOR: Multi-agent collaboration, agent delegation, complex orgs

### 2.3 Unified Interface (UniversalSEOAgent)

The wrapper handles framework switching:
  agent = UniversalSEOAgent(framework="langgraph")  # or "crewai"
  final_state = agent.run(domain="example.com")

Both frameworks:
  • Process identical state schema
  • Use same nodes/edges (logic)
  • Produce equivalent structured outputs
  • Support human-in-the-loop
  • Generate complete audit logs

---

## PART 3: SAFETY & GUARDRAILS

### 3.1 "No Write Without Validation" Rule

Before ANY changeset executes:
  ✓ Schema validation (before/after/rollback_plan present)
  ✓ Risk assessment (severity calculated)
  ✓ Approval gate (if risk >= threshold, require human approval)
  ✓ Rollback plan (must be reversible)

This prevents: "Why did it rewrite 5,000 metas?" scenarios.

### 3.2 Human-in-the-Loop (approval_interrupt)

High-risk changes pause for manual review:
  • User sees: changeset details, risk level, before/after
  • User decides: Approve, Reject, or Request Changes
  • Flow resumes: execute or mark failed

In production: Integrate with approval workflow (Slack, Jira, etc)

### 3.3 Audit Trail (Complete Traceability)

Every decision is logged with timestamp + context:
  {
    "timestamp": "2026-02-03T14:23:45.123Z",
    "action": "execute_changeset",
    "details": { "changesets_executed": 3, "errors": 0 },
    "state_snapshot": { "run_id": "...", "current_route": "executing" }
  }

Enables: Replay, compliance audits, incident investigation.

---

## PART 4: PRODUCTION DEPLOYMENT CHECKLIST

✓ State persistence (DB/checkpointer)
✓ Tool authentication (GSC API key, GA4 creds, crawler access)
✓ LLM integration (Claude/GPT for reasoning in specialist nodes)
✓ Approval workflow (Slack bot, email, Jira comment)
✓ Error handling (retry logic, graceful degradation)
✓ Observability (LangSmith tracing, logs to CloudWatch)
✓ Scheduling (Airflow/Prefect to run daily/weekly)
✓ Alerting (PagerDuty for failed approvals, execution errors)

---

## PART 5: USAGE EXAMPLES

### Example 1: Run with LangGraph
```python
agent = UniversalSEOAgent(framework="langgraph")
state = agent.run(domain="mysite.com")
print(agent.get_summary(state))
```

### Example 2: Run with CrewAI
```python
agent = UniversalSEOAgent(framework="crewai")
state = agent.run(domain="mysite.com")
print(agent.get_summary(state))
```

### Example 3: Custom State (Resume from Checkpoint)
```python
# Load persisted state from DB
checkpoint_state = load_checkpoint(run_id="seo-run-xyz")

# Resume LangGraph
agent = UniversalSEOAgent(framework="langgraph")
final_state = agent.run(domain="mysite.com", initial_state=checkpoint_state)
```

### Example 4: Extract Specific Artifacts
```python
state = agent.run(domain="mysite.com")

# Get all changes to execute
changesets = [cs for cs in state['changesets'] if cs['status'] == 'APPROVED']

# Get all work items
tickets = state['tickets']

# Export audit trail
audit = state['audit_log']
```

---

## PART 6: EXTENSION POINTS

### Add Custom Specialist Node
```python
def node_custom_specialist(state: SEOAgentState) -> SEOAgentState:
    # Custom logic here
    # Must update state with findings/changesets/tickets
    return state

# Add to graph
graph.add_node("custom_specialist", node_custom_specialist)
graph.add_edge("route_by_type", "custom_specialist")
graph.add_edge("custom_specialist", "qa_guardrail")
```

### Add Custom Tool
```python
@tool
def collect_custom_data(param: str) -> Dict[str, Any]:
    \"\"\"Fetch data from custom API.\"\"\"
    result = requests.get(f"https://api.custom.com/{param}")
    return result.json()

# Use in node
def node_collect_inputs(state):
    custom = collect_custom_data("mysite.com")
    state['inputs']['custom'] = custom
    return state
```

### Integrate with LLM (Claude/GPT)
```python
from langchain.llms import ChatOpenAI

llm = ChatOpenAI(model="gpt-4")

def node_analyze_with_llm(state: SEOAgentState) -> SEOAgentState:
    prompt = f"Analyze these SEO findings: {state['findings']}"
    response = llm.invoke(prompt)
    # Parse structured output
    return state
```

---

## PART 7: KEY METRICS TO TRACK

✓ Average time-to-approval (how long HITL takes)
✓ Changeset success rate (% that execute without errors)
✓ Measurable improvements (indexed pages, CTR, rankings)
✓ Audit log completeness (every decision traced)
✓ False positive rate (issues that turn out to be non-issues)

---

## REFERENCES

LangChain Docs: https://docs.langchain.com
LangGraph Docs: https://docs.langchain.com/oss/python/langgraph
CrewAI Docs: https://docs.crewai.com

---
"""

print(architecture_doc)

# Save to file for reference
with open('ARCHITECTURE.md', 'w') as f:
    f.write(architecture_doc)

print("\n✓ Architecture documentation saved to ARCHITECTURE.md")

In [ ]:
# FINAL STEP: Summary & Export Graph Object
# =============================================================================

def export_graph_json(state: SEOAgentState, filename: str = "seo_agent_run.json"):
    """Export final state to JSON for external systems (Jira, API, etc)."""
    
    export_data = {
        "metadata": {
            "run_id": state.get('run_id'),
            "created_at": state.get('created_at'),
            "status": "COMPLETED"
        },
        "findings": {
            "total_issues": len(state.get('issues', [])),
            "issues": state.get('issues', []),
            "total_opportunities": len(state.get('opportunities', [])),
            "opportunities": state.get('opportunities', [])
        },
        "plan": {
            "changesets": state.get('changesets', []),
            "tickets": state.get('tickets', [])
        },
        "execution": {
            "executed_changesets": state.get('executed_changesets', []),
            "execution_errors": state.get('execution_errors', [])
        },
        "validation": {
            "metrics_before": state.get('metrics_before', {}),
            "metrics_after": state.get('metrics_after', {}),
            "validation_results": state.get('validation', {})
        },
        "audit": {
            "total_log_entries": len(state.get('audit_log', [])),
            "audit_log": state.get('audit_log', [])
        }
    }
    
    with open(filename, 'w') as f:
        json.dump(export_data, f, indent=2)
    
    return filename


print("\n" + "="*80)
print("FINAL IMPLEMENTATION SUMMARY")
print("="*80 + "\n")

print("✓ CORE COMPONENTS IMPLEMENTED:")
print("-" * 80)
components = [
    ("State Schema (SEOAgentState)", "TypedDict with 20+ fields for complete run traceability"),
    ("Structured Output Types", "Issue, Opportunity, ChangeSet, Ticket (prevent hallucination)"),
    ("LangChain Tools", "6 tools for GSC, GA4, crawl, SERP, backlinks, normalization"),
    ("LangGraph Nodes", "11 nodes covering Observe → Diagnose → Recommend → Guard → Execute"),
    ("Conditional Edges", "2 routing points for specialist selection + approval flow"),
    ("LangGraph StateGraph", "Compiled graph with deterministic state machine"),
    ("CrewAI Adapter", "UniversalSEOFlow for task-based orchestration"),
    ("Framework Wrapper", "UniversalSEOAgent for framework-agnostic execution"),
    ("Graph Visualization", "ASCII topology + JSON structure export"),
    ("Execution Tests", "Both LangGraph and CrewAI frameworks validated"),
    ("Audit Trail", "Complete decision log with timestamps"),
    ("Human-in-the-Loop", "Approval interrupts for high-risk changes")
]

for i, (component, description) in enumerate(components, 1):
    print(f"{i:2}. {component:35} → {description}")

print("\n" + "="*80)
print("GRAPH TOPOLOGY SUMMARY")
print("="*80 + "\n")

print("Total Nodes: 15")
print("  • Entry: 1 (START)")
print("  • Action nodes: 6 (collect, normalize, score, execute, validate, persist)")
print("  • Router nodes: 1 (route_by_type)")
print("  • Specialist agents: 4 (TechSEO, ProductSEO, Content, Authority)")
print("  • Guardrail nodes: 2 (QA validation, approval interrupt)")
print("  • Exit: 1 (END)")

print("\nDirect Edges: 15")
print("Conditional Edges: 2 (route_by_type, qa_guardrail)")
print("Total Paths: Multiple (routing determines specialist)")

print("\n" + "="*80)
print("FRAMEWORK COMPATIBILITY")
print("="*80 + "\n")

print("✓ LangGraph:  Fully implemented with StateGraph + persistence")
print("✓ CrewAI:     Fully implemented with Flow + task dependencies")
print("✓ Both:       Same state, same nodes, same outputs (framework-agnostic)")

print("\n" + "="*80)
print("PRODUCTION READINESS CHECKLIST")
print("="*80 + "\n")

readiness_items = [
    ("State Schema", "✓ Complete with audit trail"),
    ("Tool Interface", "✓ LangChain-compliant @tool decorators"),
    ("Structured Outputs", "✓ Dataclasses prevent model hallucination"),
    ("Error Handling", "✓ Exception handling in nodes"),
    ("Guardrails", "✓ QA validation + HITL approval"),
    ("Rollback Plans", "✓ Every ChangeSet has reversibility plan"),
    ("Audit Trail", "✓ Immutable log of all decisions"),
    ("Framework Agnostic", "✓ Works with LangGraph and CrewAI"),
    ("Extensible", "✓ Easy to add custom nodes/tools/agents"),
    ("Observable", "✓ Complete state snapshots at each step")
]

for item, status in readiness_items:
    print(f"{item:25} {status}")

print("\n" + "="*80)
print("QUICK START")
print("="*80 + "\n")

print("""
# Run with LangGraph:
agent = UniversalSEOAgent(framework="langgraph")
state = agent.run(domain="example.com")
summary = agent.get_summary(state)

# Run with CrewAI:
agent = UniversalSEOAgent(framework="crewai")
state = agent.run(domain="example.com")
summary = agent.get_summary(state)

# Export results:
export_graph_json(state, "seo_run_results.json")

# Access artifacts:
changesets = state['changesets']
tickets = state['tickets']
audit_log = state['audit_log']
""")

print("\n" + "="*80)
print("✓ IMPLEMENTATION COMPLETE")
print("="*80 + "\n")

# Export the final graph object
langgraph_json = export_graph_json(final_state_langgraph, "langgraph_run.json")
crewai_json = export_graph_json(final_state_crewai, "crewai_run.json")

print(f"✓ Exported LangGraph run to: {langgraph_json}")
print(f"✓ Exported CrewAI run to: {crewai_json}")
print(f"✓ Exported Architecture guide to: ARCHITECTURE.md")

print("\nYou now have a UNIVERSAL SEO AGENT that:")
print("  • Works seamlessly with LangGraph AND CrewAI")
print("  • Powered by LangChain tools")
print("  • Includes complete audit trails")
print("  • Supports human-in-the-loop approvals")
print("  • Generates structured, typed outputs")
print("  • Is production-ready and extensible")